# LangChain with Snowflake Stage

Example based on the following resources:
* https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/
* https://quickstarts.snowflake.com/guide/asking_questions_to_your_own_documents_with_snowflake_cortex/index.html?index=..%2F..index#0

Let us define some variables:

In [1]:
MODEL_EMBEDDINGS = "e5-base-v2"
VECTOR_LENGTH = 786
CONNECTION_NAME = "arctic_user"
STAGE_NAME = "CORTEX_DB.PUBLIC.DOCUMENTS"
MODEL_LLM = "llama3-70b"

DEBUG_LANGCHAIN = False

GITHUB_ROOT = "https://github.com/Snowflake-Labs/sfquickstarts"
GITHUB_ASSETS = (
    "ask-questions-to-your-documents-using-rag-with-snowflake-cortex/assets/"
)
GITHUB_URL = f"{GITHUB_ROOT}/blob/9b1369fe6198ee62e0fc8d879209bc4c0b5b9cb6/site/sfguides/src/{GITHUB_ASSETS}"
PDF_LIST = [
    f"{GITHUB_URL}/Mondracer_Infant_Bike.pdf",
    f"{GITHUB_URL}/Premium_Bicycle_User_Guide.pdf",
    f"{GITHUB_URL}/Ski_Boots_TDBootz_Special.pdf",
    f"{GITHUB_URL}/The_Ultimate_Downhill_Bike.pdf",
    f"{GITHUB_URL}/The_Xtreme_Road_Bike_105_SL.pdf",
]

We will be connecting to Snowflake, so let us define the connection:

In [2]:
import snowflake.connector

snowflake_connection = snowflake.connector.connect(
    connection_name=CONNECTION_NAME,
)

Next, download pdf files from github and upload them to the stage

In [3]:
import requests
import os

tempdir = os.path.join(os.getcwd(), "temp_pdfs")
os.makedirs(tempdir, exist_ok=True)

with snowflake_connection.cursor() as cs:
    cs.execute(f"DROP STAGE IF EXISTS {STAGE_NAME}")
    cs.execute(f"CREATE STAGE IF NOT EXISTS {STAGE_NAME}")

    for url in PDF_LIST:
        filename = os.path.basename(url)
        local_file_name_with_path = f"{tempdir}/{filename}"

        if not os.path.exists(local_file_name_with_path):
            r = requests.get(f"{url}?raw=true", stream=True)

            with open(local_file_name_with_path, "wb") as fd:
                for chunk in r.iter_content(1024):
                    fd.write(chunk)

        cs.execute(
            f"PUT file://{local_file_name_with_path} @{STAGE_NAME}/descriptions/bikes/ auto_compress=false"
        )
        print(cs.fetchall())

[('Mondracer_Infant_Bike.pdf', 'Mondracer_Infant_Bike.pdf', 30551, 30560, 'NONE', 'NONE', 'UPLOADED', '')]
[('Premium_Bicycle_User_Guide.pdf', 'Premium_Bicycle_User_Guide.pdf', 52418, 52432, 'NONE', 'NONE', 'UPLOADED', '')]
[('Ski_Boots_TDBootz_Special.pdf', 'Ski_Boots_TDBootz_Special.pdf', 57850, 57856, 'NONE', 'NONE', 'UPLOADED', '')]
[('The_Ultimate_Downhill_Bike.pdf', 'The_Ultimate_Downhill_Bike.pdf', 60380, 60384, 'NONE', 'NONE', 'UPLOADED', '')]
[('The_Xtreme_Road_Bike_105_SL.pdf', 'The_Xtreme_Road_Bike_105_SL.pdf', 40787, 40800, 'NONE', 'NONE', 'UPLOADED', '')]


## Get a single file from Snowflake Stage

Import `SnowflakeStageFileLoader`

In [4]:
from langchain_snowpoc.document_loaders.snowflake_stage_file import (
    SnowflakeStageFileLoader,
)

Get a specific file

In [5]:
ssfl = SnowflakeStageFileLoader(
    staged_file_path=f"@{STAGE_NAME}/descriptions/bikes/The_Ultimate_Downhill_Bike.pdf",
    connection=snowflake_connection,
)
doc = ssfl.load()

> Note: Data from the pdf is extracted automatically, and a Document is returned.

Let's display content of the document (just first 500 chars):

In [6]:
from IPython.display import display, Markdown

display(Markdown(doc[0].page_content[:500] + "... AND SO ON"))

The Ultimate Downhill Bike “Rincon del Cielo”:

Downhill biking, also known as downhill mountain biking (DH), is an exhilarating and challenging sport that requires the right equipment and skills. A downhill bike, speciﬁcally designed for tackling steep, rugged terrain at high speeds, plays a vital role in your performance and safety on the trails. In this guide, we'll cover everything you need to know about downhill bikes, from understanding their components to selecting the right bike and main... AND SO ON

Great! Let's do it on a bigger scale.

## Get multiple files from Snowflake Stage

For debug purpose let us set the `langchain.debug`. Its value was defined at the beginning of this file.

In [7]:
import langchain

langchain.debug = DEBUG_LANGCHAIN

Import `SnowflakeStageDirectoryLoader` to load documents from the stage.

In [8]:
from langchain_snowpoc.document_loaders.snowflake_stage_directory import (
    SnowflakeStageDirectoryLoader,
)

ssdl = SnowflakeStageDirectoryLoader(
    stage_directory=f"@{STAGE_NAME}/descriptions/bikes/",
    connection=snowflake_connection,
)
docs = ssdl.load()

  0%|          | 0/5 [00:00<?, ?it/s]

How many documents do we have?

In [9]:
len(docs)

5

Check some metadata:

In [10]:
from pprint import pprint

for doc in docs:
    pprint(doc.metadata)

{'last_modified': 'Fri, 5 Jul 2024 10:30:37 GMT',
 'md5': 'c013a1a158ac33f280ab1f8dbe8b4813',
 'name': 'documents/descriptions/bikes/Mondracer_Infant_Bike.pdf',
 'size': 30560,
 'source': '@documents/descriptions/bikes/Mondracer_Infant_Bike.pdf'}
{'last_modified': 'Fri, 5 Jul 2024 10:30:38 GMT',
 'md5': '83ea07b38602167733967d61f2be25d7',
 'name': 'documents/descriptions/bikes/Premium_Bicycle_User_Guide.pdf',
 'size': 52432,
 'source': '@documents/descriptions/bikes/Premium_Bicycle_User_Guide.pdf'}
{'last_modified': 'Fri, 5 Jul 2024 10:30:40 GMT',
 'md5': '370b057f88f98d21681ccb684f4d0e83',
 'name': 'documents/descriptions/bikes/Ski_Boots_TDBootz_Special.pdf',
 'size': 57856,
 'source': '@documents/descriptions/bikes/Ski_Boots_TDBootz_Special.pdf'}
{'last_modified': 'Fri, 5 Jul 2024 10:30:41 GMT',
 'md5': '6b16c052fe9b48d20947ab20834dd955',
 'name': 'documents/descriptions/bikes/The_Ultimate_Downhill_Bike.pdf',
 'size': 60384,
 'source': '@documents/descriptions/bikes/The_Ultimate_Down

Now, create llm, embeddings, vector and split the documents

In [11]:
from langchain_snowpoc.llms import SQLCortex

llm = SQLCortex(connection=snowflake_connection, model=MODEL_LLM)

In [12]:
from langchain_snowpoc.embedding import SnowflakeEmbeddings

embeddings = SnowflakeEmbeddings(
    connection=snowflake_connection, model=MODEL_EMBEDDINGS
)

In [13]:
from langchain_snowpoc.vectorstores import SnowflakeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=400,
    length_function=len,
    is_separator_regex=False,
)

documents = text_splitter.split_documents(docs)

vector = SnowflakeVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    vector_length=VECTOR_LENGTH,
    connection=snowflake_connection,
)
retriever = vector.as_retriever()

Setup history support for LangChain

In [14]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

Prompt for answering questions

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

And a RAG chain:

In [16]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [17]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

A helper function, to make it easy to get answers

In [18]:
def ask(question, session_id="123"):
    return conversational_rag_chain.invoke(
        {"input": question},
        config={
            "configurable": {"session_id": session_id}
        },  # constructs a key "abc123" in `store`.
    )["answer"]

## Sample usage

Let's see some examples

### Ski boots

In [19]:
ask("What is the name of the ski boots?", session_id="ski boots")

'The name of the ski boots is TDBootz Special Ski Boots.'

In [20]:
ask("Where have they been tested?", session_id="ski boots")

'The TDBootz Ski Boots have been extensively tested in the Cerler ski resort located in northern Spain.'

In [21]:
ask("What are they good for?", session_id="ski boots")

'The TDBootz Ski Boots are the perfect choice for adventure skiers that are looking for the highest quality off-piste.'

### Downhill bike

In [22]:
ask("What is the name of the downhill bike?", session_id="downhill bike")

'The name of the downhill bike is "Rincon del Cielo".'

In [23]:
ask("What is it made of?", session_id="downhill bike")

'The Rincon del Cielo downhill bike is made of the best carbon fiber of the market, 123-Carbon-Super.'

In [24]:
ask("Who tested the bike?", session_id="downhill bike")

'The bike was tested by specialized riders, Dash, Julian, and Carlos.'

In [25]:
ask("What are they famous about?", session_id="downhill bike")

'They are famous not only for their skills with snow sports but also for their expertise in riding bikes.'

### Infant bike

In [26]:
ask("What is the max speed for the infant bike?", session_id="infant bike")

'According to the Mondracer Infant Bike user guide, the maximum speed for the infant bike is 12 miles per hour.'

In [27]:
ask("Do I need any special tool for the assembly?", session_id="infant bike")

'Yes, the Mondracer Infant bike brings a special allen wrench that needs to be used for the assembly. Not using this tool will break the warranty.'

In [28]:
ask("How should I clean the bike?", session_id="infant bike")

'You should clean the bike regularly with mild soap and water to remove dirt, grime, and debris.'